In [1]:
# Dependencies
import pandas as pd
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
import csv
import seaborn as sns
from collections import Counter

import statistics as stats

from numpy.random import seed

import seaborn as sns; sns.set()

import os

import warnings


In [4]:
#request data from json file
#loop thruogh the json to get the individual elements out of the json file and into a pandas data frame

columns = ["LEGISLATOR GOVTRACK ID", "FIRST NAME", "LAST NAME", "GENDER", "CONGRESSIONAL BRANCH", "TERM START DATE", "TERM END DATE", "STATE", "POLITICAL PARTY", "STATE DISTRICT"]

resp = requests.get('https://theunitedstates.io/congress-legislators/legislators-historical.json')
txt = resp.json()
data = []
for x in txt:
    try:
        leg_list = []
        leg_list.append(x['id']['bioguide'])
        leg_list.append(x['name']['first'])
        leg_list.append(x['name']['last']) 
        leg_list.append(x['bio']['gender'])
        leg_list.append(x['terms'][0]['type'])
        leg_list.append(x['terms'][0]['start'])
        leg_list.append(x['terms'][-1]['end'])
        leg_list.append(x['terms'][0]['state'])
        leg_list.append(x['terms'][0]['party'])
        leg_list.append(x['terms'][0]['district'])

        data.append(leg_list)
    except:
        leg_list.append(None)
legislator = pd.DataFrame(data, columns=columns)
legislator.head()

,LEGISLATOR GOVTRACK ID,FIRST NAME,LAST NAME,GENDER,CONGRESSIONAL BRANCH,TERM START DATE,TERM END DATE,STATE,POLITICAL PARTY,STATE DISTRICT
0,S000349,Roger,Sherman,M,rep,1789-03-04,1793-12-31,CT,Pro-Administration,-1
1,W000551,Hugh,Williamson,M,rep,1789-03-04,1793-03-03,NC,Federalist,2
2,D000157,William,Dawson,M,rep,1793-12-02,1795-03-03,NC,Anti-Administration,8
3,V000104,John,Vining,M,rep,1789-03-04,1799-03-03,DE,Pro-Administration,0
4,A000174,Fisher,Ames,M,rep,1789-03-04,1797-03-03,MA,Federalist,1


In [6]:
legislator.shape

(10499, 10)

In [7]:
#check the the missing values
legislator.isna().sum()

LEGISLATOR GOVTRACK ID    0
FIRST NAME                0
LAST NAME                 0
GENDER                    0
CONGRESSIONAL BRANCH      0
TERM START DATE           0
TERM END DATE             0
STATE                     0
POLITICAL PARTY           0
STATE DISTRICT            0
dtype: int64

In [8]:
# Check Unique 	LEGISLATOR GOVTRACK ID

id_count = legislator["LEGISLATOR GOVTRACK ID"].nunique()
print(id_count)

10499


In [9]:
# Check for duplicates on LEGISLATOR GOVTRACK ID
legislator.loc[legislator[['LEGISLATOR GOVTRACK ID']].duplicated(keep='first'), :]

,LEGISLATOR GOVTRACK ID,FIRST NAME,LAST NAME,GENDER,CONGRESSIONAL BRANCH,TERM START DATE,TERM END DATE,STATE,POLITICAL PARTY,STATE DISTRICT


In [10]:
#add a column for term start year and term end year

legislator['TERM START YEAR'] = legislator['TERM START DATE'].astype(str).str[:4]
legislator['TERM END YEAR'] = legislator['TERM END DATE'].astype(str).str[:4]

In [11]:
#add a column for term start year and term end year
legislator['TERM START YEAR'].astype(int)
legislator['TERM END YEAR'].astype(int)

0        1793
1        1793
2        1795
3        1799
4        1797
         ... 
10494    2020
10495    2020
10496    2020
10497    2020
10498    2020
Name: TERM END YEAR, Length: 10499, dtype: int32

In [12]:
#create a new column for congressional district as well
legislator['CONGRESSIONAL DISTRICT'] = legislator['STATE'].astype(str) + legislator['STATE DISTRICT'].astype(str)

In [13]:
#lets look at our data frame and see how it is coming along
legislator.head()

,LEGISLATOR GOVTRACK ID,FIRST NAME,LAST NAME,GENDER,CONGRESSIONAL BRANCH,TERM START DATE,TERM END DATE,STATE,POLITICAL PARTY,STATE DISTRICT,TERM START YEAR,TERM END YEAR,CONGRESSIONAL DISTRICT
0,S000349,Roger,Sherman,M,rep,1789-03-04,1793-12-31,CT,Pro-Administration,-1,1789,1793,CT-1
1,W000551,Hugh,Williamson,M,rep,1789-03-04,1793-03-03,NC,Federalist,2,1789,1793,NC2
2,D000157,William,Dawson,M,rep,1793-12-02,1795-03-03,NC,Anti-Administration,8,1793,1795,NC8
3,V000104,John,Vining,M,rep,1789-03-04,1799-03-03,DE,Pro-Administration,0,1789,1799,DE0
4,A000174,Fisher,Ames,M,rep,1789-03-04,1797-03-03,MA,Federalist,1,1789,1797,MA1


In [14]:
#this data frame has data from the 1700s....lets filter the data for the years that we need
legislator_df = legislator[(legislator['TERM END YEAR'] < '1960')]
legislator_df

,LEGISLATOR GOVTRACK ID,FIRST NAME,LAST NAME,GENDER,CONGRESSIONAL BRANCH,TERM START DATE,TERM END DATE,STATE,POLITICAL PARTY,STATE DISTRICT,TERM START YEAR,TERM END YEAR,CONGRESSIONAL DISTRICT
0,S000349,Roger,Sherman,M,rep,1789-03-04,1793-12-31,CT,Pro-Administration,-1,1789,1793,CT-1
1,W000551,Hugh,Williamson,M,rep,1789-03-04,1793-03-03,NC,Federalist,2,1789,1793,NC2
2,D000157,William,Dawson,M,rep,1793-12-02,1795-03-03,NC,Anti-Administration,8,1793,1795,NC8
3,V000104,John,Vining,M,rep,1789-03-04,1799-03-03,DE,Pro-Administration,0,1789,1799,DE0
4,A000174,Fisher,Ames,M,rep,1789-03-04,1797-03-03,MA,Federalist,1,1789,1797,MA1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8366,V000125,Charles,Vursell,M,rep,1943-01-06,1959-01-03,IL,Republican,23,1943,1959,IL23
8367,W000450,Richard,Wigglesworth,M,rep,1927-12-05,1959-01-03,MA,Republican,14,1927,1959,MA14
8368,W000549,William,Williams,M,rep,1951-01-03,1959-01-03,NY,Republican,35,1951,1959,NY35
8369,W000683,Charles,Wolverton,M,rep,1927-12-05,1959-01-03,NJ,Republican,1,1927,1959,NJ1


In [17]:
#lets save this pandas dataframe as a csv file so we can import it into our database
legislator_df.to_csv(r'C:/Users/Brian/Desktop/ETL_Project_Gun_Violence-MS_Database/legislator_2017_df.csv', index = False)